In [2]:
# stdlib
from pathlib import Path
from typing import List,Any,Dict

import pandas as pd

from sqlalchemy import create_engine, select, inspect, and_, exists
from sqlalchemy.orm import Session

import sys
cwd = Path.cwd().parent.joinpath("money_map")
sys.path.append(cwd)

# custom class
from money_map.models.orm_models import (Transactions_Labeled_Table)
from money_map.pipelines.transactions import recreate_transactions_labeled
from money_map.connect.mysql_conector import MySQLConnector


connector = MySQLConnector()
engine = connector.create_sql_engine()
recreate_transactions_labeled(engine=engine)

In [5]:

target_account:str = "GB87YODA19011530005979"
with Session(engine) as session:
    stmt = (select(Transactions_Labeled_Table)
            .filter(Transactions_Labeled_Table.sender_iban==target_account))
    df = pd.read_sql(stmt, session.bind)


df = df.sort_values(by="booking_date")

# define day, month, year based on booking_date
df["booking_date"] = pd.to_datetime(df["booking_date"],format="%Y-%m-%d")
df["value_date"] = pd.to_datetime(df["value_date"],format="%Y-%m-%d")
df["day"] = df["booking_date"].dt.day
df["month"] = df["booking_date"].dt.month
df["year"] = df["booking_date"].dt.year

print(list(df))
# df.to_csv(r"C:\Users\hanne\Documents\GitHub\money-map\data\protected\rbpn_transactions_labeled_backup.csv")
df.head(2)

['transaction_id', 'sender_account_type', 'sender_iban', 'sender_bic', 'sender_bank_name', 'receiver_name', 'receiver_iban', 'receiver_bic', 'booking_date', 'value_date', 'amount', 'currency', 'booking_text', 'purpose', 'balance_after_booking', 'notes', 'default_category', 'tax_relevant', 'creditor_id', 'mandate_reference', 'purpose_char', 'category_id', 'category_1', 'category_2', 'category_3', 'day', 'month', 'year']


,transaction_id,sender_account_type,sender_iban,sender_bic,sender_bank_name,receiver_name,receiver_iban,receiver_bic,booking_date,value_date,amount,currency,booking_text,purpose,balance_after_booking,notes,default_category,tax_relevant,creditor_id,mandate_reference,purpose_char,category_id,category_1,category_2,category_3,day,month,year
0,1530005979_01012024_29815_Tree red,Girokonto,GB87YODA19011530005979,OECF10873187191890,RBPN,Andrew Myers,GB25CZXL23186024805804,XQAB67042397904377,2024-01-01,2024-01-01,-184.81,EUR,DAUERAUFTRAG,Tree red present pick paper.,29815.2,None,None,None,None,None,tree red present pick paper,16,Fixed expenses,Insurance,Supplementary health insurance,1,1,2024
15,1530005979_07012024_29059_Recent s,Girokonto,GB87YODA19011530005979,OECF10873187191890,RBPN,Ashley Stewart,GB30LIPU43896807588812,VWNK21811988757283,2024-01-07,2024-01-07,-228.55,EUR,LASTSCHRIFT,Recent stand wrong interest authority character.,29059.7,None,None,None,None,None,recent stand wrong interest authority character,50,Variable expenses,Entertainment,Going out,7,1,2024



# Monthly


In [6]:
# define monthly dataset
ignore_categories_3 = ["Transfers"]

monthly_df = df.loc[~df["category_3"].isin(ignore_categories_3)]
monthly_df = (monthly_df.groupby(["year","month","category_id"])
                        .agg({"amount":"sum","balance_after_booking":"mean","category_1":"first","category_2":"first","category_3":"first"})
                        .reset_index())

monthly_df["year-month"] = monthly_df["year"].astype(str) + "-" + monthly_df["month"].astype(str)
monthly_df_total = monthly_df[["year-month","amount"]].groupby('year-month').sum()
monthly_df_total["amount"] = monthly_df_total["amount"].astype(int)

In [7]:
import plotly.express as px
import plotly.graph_objects as go

fig = px.bar(monthly_df,x="year-month",y="amount",color="category_3",height=1000)
fig.add_trace(go.Scatter(
    x=monthly_df_total .index,
    y=monthly_df_total['amount'],
    text=monthly_df_total ['amount'],
    mode='text',
    textposition='top center',
    textfont=dict(
        size=18,
    ),
    showlegend=False
))
fig.update_layout(title=f'Transactions: {target_account}')
fig.show()

In [8]:
daily_df = (df.groupby(["booking_date"])
                .agg(min_balance_after_booking=("balance_after_booking","min"),
                     mean_balance_after_booking=("balance_after_booking","mean"),
                     max_balance_after_booking=("balance_after_booking","max"))
                .reset_index())

fig = go.Figure()
fig.add_trace(go.Scatter(x=daily_df["booking_date"],y=daily_df["min_balance_after_booking"],mode='lines+markers',name="min_balance_after_booking"))
fig.add_trace(go.Scatter(x=daily_df["booking_date"],y=daily_df["max_balance_after_booking"],mode='lines+markers',name="max_balance_after_booking"))
fig.update_layout(title=f'Balance: {target_account}',
                  xaxis_title='Booking Date',
                  yaxis_title='Balance [€]')
fig.show()